In [326]:
import requests
import json

In [331]:
# Initialize API_KEY
API_KEY = ""
# Initialize collection slugs
slugs = [
#     "anticyclone-by-william-mapan",
#     "flux-by-owen-moore",
#     "ancient-courses-of-fictional-rivers-by-robert-hodg",
#     "memories-of-qilin-by-emily-xie",
#     "entretiempos-by-marcelo-soria-rodriguez",
#     "para-bellum-by-matty-mariansky",
#     "screens-by-thomas-lin-pedersen",
#     "cosmic-reef-by-leo-villareal",
#     "chimera-by-mpkoz",
#     "jiometory-no-compute-ziometori-ha-keisansaremasen",
#     "vortex-by-jen-stark",
#     "gazers-by-matt-kane",
#     "bent-by-ippsketch",
#     "autology-by-steganon",
#     "asemica-by-emily-edelman-dima-ofman-andrew-badr",
#     "edifice-by-ben-kovach",
#     "skulptuur-by-piter-pasma",
#     "fragments-of-an-infinite-field-by-monica-rizzolli",
#     "trossets-by-anna-carreras",
#     "geometry-runners-by-rich-lord-1",
#     "phase-by-loren-bednar",
#     "scribbled-boundaries-by-william-tan",
#     "pigments-by-darien-brito",
#     "endless-nameless-by-rafael-rozendaal",
#     "glitch-crystal-monsters-by-alida-sun",
#     "century-by-casey-reas",
#     "dreams-by-joshua-bagley",
#     "the-blocks-of-art-by-shvembldr",
#     "frammenti-by-stefano-contiero",
#     "algorhythms-by-han-x-nicolas-daniel",
#     "bubble-blobby-by-jason-ting",
#     "watercolor-dreams-by-numbersinmotion",
#     "subscapes-by-matt-deslauriers",
#     "elementals-by-michael-connolly",
#     "algobots-by-stina-jones",
#     "synapses-by-chaosconstruct",
#     "aerial-view-by-dalenz",
#     "inspirals-by-radix",
#     "apparitions-by-aaron-penne",
#     "720-minutes-by-alexis-andre",
#     "archetype-by-kjetil-golid",
#     "27-bit-digital-by-kai",
#     "spectron-by-simon-de-mai",
#     "unigrids-by-zeblocks",
#     "hyperhash-by-beervangeer",
#     "nimbuds-by-bryan-brinkman",
#     "ignition-by-ge1doot",
#     "singularity-by-hideki-tsukamoto",
#     "elevated-deconstructions-by-luxpris",
#     "cryptoblots-by-daim-aggott-honsch",
#     "dynamic-slices-by-pxlq-1",
#     "chromie-squiggle-by-snowfro",
#     "genesis-by-dca",
#     "construction-token-by-jeff-davis"
#     "ringers-by-dmitri-cherniak",
#     "fidenza-by-tyler-hobbs"
]

In [332]:
# Iterate through collection slugs
for slug in slugs:
    # Query general collection data
    url = f"https://api.opensea.io/api/v1/collection/{slug}/stats"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)

    data = json.loads(response.text)
    
    # Record current floor price
    floor_price = data["stats"]["floor_price"]
    # Record thirty-day average sale price
    thirty_day_avg = data["stats"]["thirty_day_average_price"]
    
    # Initialize sale price history array
    prices = []

    # Query sale history
    url = f"https://api.opensea.io/api/v1/events?only_opensea=true&collection_slug={slug}&event_type=successful"

    headers = {
        "Accept": "application/json",
        "X-API-KEY": f"{API_KEY}"
    }

    response = requests.get(url, headers=headers)

    data = json.loads(response.text)

    # Iterate through first page sales
    for index, sale in enumerate(data["asset_events"]):
        if data["asset_events"][index]["event_type"] == "successful":
            try:
                event_time = data["asset_events"][index]["event_timestamp"]
                eth_price = int(data["asset_events"][index]["total_price"].split(".")[0])/1000000000000000000
                usd_price = eth_price * int(data["asset_events"][index]["payment_token"]["usd_price"].split(".")[0])
            except:
                continue
        prices.append([event_time, eth_price, usd_price])
        print(f"New {slug} sale @ {event_time}: {eth_price} ETH (${usd_price})")

    # Query next page, and iterate through sales again
    while data["next"] != None:
        url = f"https://api.opensea.io/api/v1/events?only_opensea=true&collection_slug={slug}&event_type=successful&cursor={data['next']}"

        headers = {
            "Accept": "application/json",
            "X-API-KEY": f"{API_KEY}"
        }

        response = requests.get(url, headers=headers)

        data = json.loads(response.text)

        # Iterate through sales
        for index, sale in enumerate(data["asset_events"]):
            if data["asset_events"][index]["event_type"] == "successful":
                try:
                    event_time = data["asset_events"][index]["event_timestamp"]
                    eth_price = int(data["asset_events"][index]["total_price"].split(".")[0])/1000000000000000000
                    usd_price = eth_price * int(data["asset_events"][index]["payment_token"]["usd_price"].split(".")[0])
                except:
                    continue
            prices.append([event_time, eth_price, usd_price])
            print(f"New {slug} sale @ {event_time}: {eth_price} ETH (${usd_price})")
            
    # Organize json data
    metadata= {
            "floor_price" : f"{floor_price}",
            "thirty_day_avg" : f"{thirty_day_avg}",
            "price_history" : prices,
        }

    # Save price history into json file
    with open(f"price_histories/{slug}.json", "w", encoding="utf-8") as outfile:
        json.dump(metadata, outfile, ensure_ascii=False, indent=4)